# Данные

In [75]:
import pandas as pd
import numpy as np
import collections
import nltk
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
import fasttext
import pymorphy2
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [7]:
!pip install fasttext

  Stored in directory: C:\Users\Bagi\AppData\Local\pip\Cache\wheels\73\8e\5d\ecb50b90adaab5868ae1d8df180f31e55e85c2f055aaf2fb35
Successfully built fasttext


You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [8]:
!pip install pymorphy2

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
!pip install pymystem3

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
df = pd.read_csv('vk_texts_with_sources.csv', usecols = ['text', 'source'])
df.head()

,text,source
0,Начальник Главного оперативного управления Ген...,mil
1,Артиллерийские подразделения общевойскового об...,mil
2,Подразделения морской пехоты Каспийской флотил...,mil
3,Команды на всеармейских этапах конкурсов АрМИ-...,mil
4,На большом учебно-методическом командирском сб...,mil


# Домашнее задание

В этом домашнем задании вы будете решать задачу тематической классификации. Даны тексты, опубликованные в нескольких пабликах VK.com, посвященных государственным и муниципальным службам. Формально задача заключается в том, чтобы по тексту ($d$) определить в каком паблике он опубликован, то есть, к какому классу $c$ он принадлежит. 

## Задание 1 [1 балл]. Описательные статистики
Посчитайте:
* количество текстов и количество классов
* количество слов (без лемматизации и с лемматизацией) в коллекции
* среднюю длину текста в словах и символах
* найдите 5 самых частых существительных в текстах каждого паблика 

*Рекомендуем использовать pandas для расчета описательных статистик.*

Разделите коллекцию текстов на обучающую и тестовую части. 

In [12]:
print('Всего текстов :', df.shape[0])
print('Пустые тексты :', df['text'].isna().sum())
print('Уникальные тексты :', df['text'].nunique())
#убираем пустые и с дубликатами
df = df.dropna().drop_duplicates()

Всего текстов : 11625
Пустые тексты : 99
Уникальные тексты : 11149


In [13]:
print('Всего классов:', df['source'].nunique())

Всего классов: 4


In [14]:
words_bag = set()
df['text'].apply(lambda text: words_bag.update(set(nltk.tokenize.word_tokenize(text))))    
print('Количество слов (без лемматизации) в коллекции:', len(words_bag))

Количество слов (без лемматизации) в коллекции: 96746


In [35]:
m = Mystem(mystem_bin='C://Users/Bagi/MachineLearning/mystem.exe')

In [51]:
lemmatize_words_bag = set()
for word in words_bag:
lemmatize_words_bag.add(m.lemmatize(word)[0])
print('Количество слов (c лемматизацией) в коллекции:', len(lemmatize_words_bag))    

Количество слов (c лемматизацией) в коллекции: 31489


In [37]:
print('Средняя длина текста в словах:', np.mean([len(nltk.tokenize.word_tokenize(text)) for text in df['text']]))
print('Средняя длина текста в символах:', np.mean([len(text) for text in df['text']]))

Средняя длина текста в словах: 122.69765982246929
Средняя длина текста в символах: 711.1856899488927


In [52]:
morph_analyzer = pymorphy2.MorphAnalyzer()
for source in df['source'].unique():
    counter = collections.Counter()
    for text in df[df['source'] == source]['text']:
        for word in nltk.tokenize.word_tokenize(text): 
            parsed = morph_analyzer.parse(word)[0]
            if 'NOUN' in parsed.tag:
                counter.update([parsed.normal_form])
    print('Топ 5 самых частых слов', source, ':', ', '.join([word[0] for word in counter.most_common(5)]))

Топ 5 самых частых слов mil : минобороны, военный, россия, год, оборона
Топ 5 самых частых слов mchsgov : мчс, мчсроссия, россия, спасатель, человек
Топ 5 самых частых слов russianpost : почта, россия, год, отделение, письмо
Топ 5 самых частых слов mospolice : полиция, мвд, москва, россия, сотрудник


In [59]:
#предобработка текста, оставляем слова из символов русского и латинского алфавитов
def text_transform(text: pd.Series) -> pd.Series:
    return text.str.lower().replace("[^а-яА-Яa-zA-Z0-9]", " ", regex=True)
def lemmatize(text, mystem=m):
    return "".join(m.lemmatize(text)).strip()

In [149]:
df_transformed = df.copy()
df_transformed.text = df.apply(text_transform)
df_transformed.text  = df_transformed.text.apply(lemmatize)
X = df_transformed.drop('source', axis=1)
y = df_transformed['source']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

 ## Задание 2 [2 балла]. Классификация по правилам
 
 * Разработайте несколько правил вида "Если встречается слово $w$, то текст относится к паблику $c$"
 * Посчитайте, какую точность, полноту, $f$-меру и $accuracy$ вы получаете при классификации по правилам
 * Получилось ли у вас придумать правило, которое никогда не ошибается?

## Задание 3 [3 балла]. Baseline
Используйте стандартный ```sklearn.pipeline``` для классификации текстов: 
* векторизация 
* $tf-idf$ взвешивание 
* ваш любимый метод классификации.


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [63]:
pipe = Pipeline([('tfidf', TfidfVectorizer(ngram_range = (1, 2), max_df=0.95)), ('lgb', LGBMClassifier())])
pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])   

In [69]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

accuracy_score: 0.9731086410899964
classification_report:
               precision    recall  f1-score   support

     mchsgov       0.97      0.96      0.97       705
         mil       0.99      0.98      0.98       741
   mospolice       0.98      0.98      0.98       711
 russianpost       0.96      0.96      0.96       632

   micro avg       0.97      0.97      0.97      2789
   macro avg       0.97      0.97      0.97      2789
weighted avg       0.97      0.97      0.97      2789

confusion_matrix:
 [[680   6   0  19]
 [ 12 729   0   0]
 [  2   1 699   9]
 [  9   4  13 606]]


показатели качества достаточно высоки, алгоритм не только почти во всех случаях правильно отгадывает класс, но и редко присваивает неправильный класс не тому тексту.
например, тексты 3 класса только 13 раз были обозначены как 4 класса, т.е с 1 и 2 классами они хорошо разделимы. Также тексты 4 темы, не были ни разу обозначены 2 темой

## Задание 4 [2 балла]. Снижение размерности
Добавьте в ваш ```sklearn.pipeline```  методы снижения размерности:  PCA / LSI / LSA / LDA / другое. Какие методы классификации разумно использовать после снижения размерности? Как изменились результаты классификации после добавления нового шага?

In [73]:
pipe = Pipeline([('tfidf', TfidfVectorizer(ngram_range = (1, 2), max_df=0.95)),
                 ('lsa', TruncatedSVD(n_components=6)),
                 ('lgb', LGBMClassifier())])
pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])

In [74]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

accuracy_score: 0.951954105414127
classification_report:
               precision    recall  f1-score   support

     mchsgov       0.94      0.91      0.93       705
         mil       0.96      0.97      0.96       741
   mospolice       0.97      0.99      0.98       711
 russianpost       0.93      0.94      0.94       632

   micro avg       0.95      0.95      0.95      2789
   macro avg       0.95      0.95      0.95      2789
weighted avg       0.95      0.95      0.95      2789

confusion_matrix:
 [[641  26   4  34]
 [ 20 718   0   3]
 [  3   1 702   5]
 [ 15   5  18 594]]


После методов снижения размерности следует использовать алгоритмы требовательные к памяти и долгие по времени обучения, т.к количество признаков уменьшится и данные параметры также сократятся
алгоритмы, не допускающие корреляции признаков
Результаты классификации показывают снижение качества, алгоритм стал больше ошибаться и относить объект к неправильному классу. Тяжелее оценить, какие темы лучше разделимы, но видно что количество неправильно отнесенных к 1 классу объектов 4 выросло почти в 2 раза, тогда как объекты 2 класса неправильно отнесенные к 1 почти в 5 раз.

## Задание 5 [1 балл]. Лемматизация
Посмотрите, как влияет лемматизация на качество классификации. Как изменится качество классификации, если вы используете ```CountVectorizer``` на словах или $n$-граммах на лемматизированных текстах?

In [81]:
pipe = Pipeline([('tfidf', CountVectorizer(ngram_range = (1, 2))),
                 ('lsa', TruncatedSVD(n_components=6)),
                 ('lgb', LGBMClassifier())])
pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])

In [82]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

accuracy_score: 0.9268555037647902
classification_report:
               precision    recall  f1-score   support

     mchsgov       0.89      0.88      0.89       705
         mil       0.95      0.96      0.95       741
   mospolice       0.97      0.98      0.97       711
 russianpost       0.90      0.88      0.89       632

   micro avg       0.93      0.93      0.93      2789
   macro avg       0.93      0.93      0.93      2789
weighted avg       0.93      0.93      0.93      2789

confusion_matrix:
 [[622  24   6  53]
 [ 21 710   1   9]
 [  9   5 694   3]
 [ 48  11  14 559]]


In [90]:
pipe = Pipeline([('tfidf', CountVectorizer(dtype = np.float64, ngram_range = (1, 2))),
                 ('lgb', LGBMClassifier())])
pipe.fit(X_train['text'], y_train)
y_pred = pipe.predict(X_test['text'])

In [91]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

accuracy_score: 0.9741842954463965
classification_report:
               precision    recall  f1-score   support

     mchsgov       0.98      0.96      0.97       705
         mil       0.98      0.99      0.99       741
   mospolice       0.98      0.98      0.98       711
 russianpost       0.95      0.97      0.96       632

   micro avg       0.97      0.97      0.97      2789
   macro avg       0.97      0.97      0.97      2789
weighted avg       0.97      0.97      0.97      2789

confusion_matrix:
 [[679   7   0  19]
 [  9 731   0   1]
 [  3   1 696  11]
 [  4   4  13 611]]


Качество резко снизилось, если использовать понижение размерности и countvectorizer на нграммах и стало максимальным без него

## Задание 6 [3 балла]. Continious bag of words
Для baseline решения мы использовали обычное представление текста в виде мешка слов. Попробуйте использовать другие модели представления текста – например, в виде непрерывного мешка слов, то есть, в виде набора эмбеддингов. Для того, чтобы получить вектор текста попробуйте:
* усреднить все эмбеддинги слов, входящих в этот текст
* усреднить все эмбеддинги слов, входящих в этот текст с $tf-idf$ весами
* использовать любую модель эмбеддинга документа.

Используйте любую модель эмбеддингов по вашему вкусу. 


Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [110]:
texts = [pd.concat([X_train, X_test])['text'].iloc[i].split() for i in range(len(df))]
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
doc2d = Doc2Vec(documents, vector_size=100, window=5, min_count=5, workers=4)
train_doc = []
for text in texts[:X_train.shape[0]]:
    train_doc.append(model.infer_vector(texts[0]))
test_doc = []
for text in texts[X_train.shape[0]:]:
    test_doc.append(model.infer_vector(texts[0]))

In [93]:
pipe = Pipeline([('lgb', LGBMClassifier())])
pipe.fit(train_doc, y_train)
y_pred = pipe.predict(test_doc)

In [94]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

accuracy_score: 0.25170311939763357
classification_report:
               precision    recall  f1-score   support

     mchsgov       0.26      0.24      0.25       705
         mil       0.26      0.34      0.29       741
   mospolice       0.27      0.26      0.26       711
 russianpost       0.20      0.15      0.17       632

   micro avg       0.25      0.25      0.25      2789
   macro avg       0.25      0.25      0.25      2789
weighted avg       0.25      0.25      0.25      2789

confusion_matrix:
 [[172 227 176 130]
 [186 250 181 124]
 [154 258 186 113]
 [153 232 153  94]]


Doc2vec проявил себя не лучшим образом, малая точность и полнота, выбирая наугад 1 из 4 классов, качество будет похожим

In [200]:
word2v = Word2Vec(texts, size=100, window=5, min_count=5, workers=4)
tf = TfidfVectorizer(token_pattern=r'\b\w+\b')
tf.fit(X_train['text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='\\b\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [171]:
def get_text(data, tfidf=None):
    emb = []
    for text in data['text']:
        text_emb = []
        for word in text:
            try:
                if tfidf is None:
                    text_emb.append(word2v.wv.get_vector(word))
                else:
                    text_emb.append(word2v.wv.get_vector(word) * tfidf.vocabulary_[word])
            except KeyError:
                pass
        emb.append(np.mean(text_emb, axis=0))
    return emb

In [180]:
train_word = get_text(X_train)
test_word = get_text(X_test)

In [181]:
pipe = Pipeline([('lgb', LGBMClassifier())])
pipe.fit(train_word, y_train)
y_pred = pipe.predict(test_word)

In [182]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

accuracy_score: 0.7970598780925062
classification_report:
               precision    recall  f1-score   support

     mchsgov       0.86      0.78      0.82       705
         mil       0.81      0.86      0.83       741
   mospolice       0.81      0.82      0.81       711
 russianpost       0.70      0.72      0.71       632

   micro avg       0.80      0.80      0.80      2789
   macro avg       0.80      0.79      0.79      2789
weighted avg       0.80      0.80      0.80      2789

confusion_matrix:
 [[551  31  37  86]
 [ 27 637  36  41]
 [ 22  44 582  63]
 [ 40  75  64 453]]


In [203]:
train_word_tdf = get_text(X_train, tfidf=tf)
test_word_tdf = get_text(X_test, tfidf=tf)

In [204]:
pipe = Pipeline([('lgb', LGBMClassifier())])
pipe.fit(train_word_tdf, y_train)
y_pred = pipe.predict(test_word_tdf)

In [205]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

accuracy_score: 0.7845105772678379
classification_report:
               precision    recall  f1-score   support

     mchsgov       0.84      0.79      0.81       705
         mil       0.78      0.81      0.79       741
   mospolice       0.79      0.81      0.80       711
 russianpost       0.73      0.72      0.73       632

   micro avg       0.78      0.78      0.78      2789
   macro avg       0.78      0.78      0.78      2789
weighted avg       0.79      0.78      0.78      2789

confusion_matrix:
 [[554  35  37  79]
 [ 29 601  69  42]
 [ 20  68 579  44]
 [ 59  69  50 454]]


Ситуация с Word2vec выглядит намного более оптимистичной, предсказание на основании близлежащих слов и одновременно этого набора слов на основании одного
Из матрицы ошибок так же видно, что 4 тема оказалась менее доступной для алгоритма, ее больше всего он путал с другими темами, больше всего с первой.
В связке с tfidf vectorizer качество несильно уменьшилось

## Задание 7 [2 балла]. fastText

Используйте ```fastText``` в режиме классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

In [206]:
rows = []
for idx, text in enumerate(X_train['text']):
    rows.append(text + ' __label__' + y_train.iloc[idx])
np.savetxt('data.train.txt', rows, fmt='%s', encoding='utf-8')

Здесь происходит что-то странное с кодировкой, файл model.bin создается не в той кодировке, а bin.vec отсутствует в принципе, по итогу ни одного предсказания. Данную штуку пофиксить не удалось(

In [229]:
classifier = fasttext.supervised('data.train.txt', 'model', encoding='utf-8')
#y_pred = [cls[0] for cls in classifier.predict(X_test['text'])]
#for cls in classifier.predict(X_test['text']):
#    print(cls[0])
classifier.predict(X_test['text'])

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [ ]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('classification_report:\n', classification_report(y_test, y_pred))
print('confusion_matrix:\n', confusion_matrix(y_test, y_pred))

## Задание 8 [4 балла]. CNN

Реализуйте модель Kim et al (2014) для решения задачи классификации с помощью CNN. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).
Ссылка: Kim Y. Convolutional Neural Networks for Sentence Classification. 2014

In [3]:
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Input, MaxPooling1D, Convolution1D, Embedding
from keras.layers.merge import Concatenate
from keras.preprocessing import sequence
from keras.utils import to_categorical

batch_size = 64
num_epochs = 10
emb_size = 100
sequence_length = 50


def build_model(w2v_model):
    model_input = Input(shape=(sequence_length,))
    x = Embedding(len(w2v_model.wv.vocab), emb_size, input_length=sequence_length, name="embedding")(model_input)
    x = Dropout(0.2)(x)

    conv_blocks = []
    for sz in [3, 7]:
        conv = Convolution1D(filters=10, kernel_size=sz,  padding="valid", activation="relu", strides=1)(x)
        conv = MaxPooling1D()(conv)
        conv = Flatten()(conv)
        conv_blocks.append(conv)

    x = Concatenate()(conv_blocks)

    x = Dropout(0.3)(x)
    x = Dense(50, activation="relu")(x)
    model_output = Dense(4, activation="softmax")(x)

    model = Model(model_input, model_output)
    
    # Initialize weights with word2vec
    weights = np.array([w2v_model.wv.get_vector(word) for word in w2v_model.wv.vocab.keys()])
    embedding_layer = model.get_layer("embedding")
    embedding_layer.set_weights([weights])
    return model


w2v_model = Word2Vec(texts, size=emb_size, window=5, min_count=1, workers=4)
model = build_model(w2v_model)
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])"""

In [272]:
word_to_idx = {key:idx for idx, key in enumerate(word2v.wv.vocab.keys())}

In [273]:
x_train = sequence.pad_sequences([[word_to_idx[word] for word in text.split()] for text in X_train['text']], 
                                   maxlen=sequence_length, padding="post", truncating="post")
x_test = sequence.pad_sequences([[word_to_idx[word] for word in text.split()] for text in X_test['text']], 
                                  maxlen=sequence_length, padding="post", truncating="post")

In [274]:
labels = {'mil': 0, 'mchsgov': 1, 'russianpost': 2, 'mospolice': 3}
inv_labels = {0: 'mil', 1: 'mchsgov', 2: 'russianpost', 3: 'mospolice'}

In [275]:
model.fit(x_train, to_categorical([labels[y] for y in y_train], num_classes=4),
          validation_data=(x_test, to_categorical([labels[y] for y in y_test], num_classes=4),),
          batch_size=batch_size, epochs=num_epochs, verbose=2)

Train on 8364 samples, validate on 2789 samples
Epoch 1/10
 - 10s - loss: 0.8681 - acc: 0.8754 - val_loss: 0.2627 - val_acc: 0.9165
Epoch 2/10
 - 9s - loss: 0.1446 - acc: 0.9585 - val_loss: 0.1746 - val_acc: 0.9520
Epoch 3/10
 - 9s - loss: 0.1047 - acc: 0.9694 - val_loss: 0.1513 - val_acc: 0.9624
Epoch 4/10
 - 9s - loss: 0.0780 - acc: 0.9766 - val_loss: 0.1447 - val_acc: 0.9602
Epoch 5/10
 - 9s - loss: 0.0627 - acc: 0.9825 - val_loss: 0.1615 - val_acc: 0.9588
Epoch 6/10
 - 9s - loss: 0.0518 - acc: 0.9831 - val_loss: 0.1533 - val_acc: 0.9616
Epoch 7/10
 - 9s - loss: 0.0468 - acc: 0.9849 - val_loss: 0.1605 - val_acc: 0.9620
Epoch 8/10
 - 9s - loss: 0.0383 - acc: 0.9891 - val_loss: 0.2355 - val_acc: 0.9175
Epoch 9/10
 - 9s - loss: 0.0348 - acc: 0.9904 - val_loss: 0.1517 - val_acc: 0.9634
Epoch 10/10
 - 9s - loss: 0.0302 - acc: 0.9903 - val_loss: 0.1501 - val_acc: 0.9613


Сетка быстро обучалась, сильно наращивая за первые шаги все метрики качества

## Задание 9 [4 + 2 балла]. RNN

(4 балла)Используйте ```RNN``` (BLSTM с какими-то признаками и пулинг поверх) для решения задачи текстовой классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix).

За дополнительные 2 балла добавьте в модель символьные признаки - CharCNN или CharRNN

In [296]:
from keras.layers import Embedding, LSTM, Dense, TimeDistributed, Dropout, Bidirectional, Input, concatenate
from keras.models import Model

word2v = Word2Vec(texts, size=emb_size, window=5, min_count=1, workers=4)

batch_size = 128
num_epochs = 10
emb_size = 100
sequence_length = 50

def build_model(word2v):
    weights = np.array([word2v.wv.get_vector(word) for word in word2v.wv.vocab.keys()])
    model_input_sent = Input(shape=(sequence_length,))
    x = Embedding(len(word2v.wv.vocab), emb_size, 
                  input_length=sequence_length, weights=[weights], 
                  name="embedding")(model_input_sent)
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = MaxPooling1D()(x)
    x = Dropout(0.2)(x)
    x = Flatten()(x)
    x = Dense(4, activation="softmax")(x)
    model = Model(inputs=[model_input_sent], outputs=x)
    return model
model2 = build_model(word2v)
model2.compile(loss="categorical_crossentropy", optimizer="Adagrad", metrics=["accuracy"])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 50)                0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 100)           3550500   
_________________________________________________________________
bidirectional_8 (Bidirection (None, 50, 128)           84480     
_________________________________________________________________
max_pooling1d_55 (MaxPooling (None, 25, 128)           0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 25, 128)           0         
_________________________________________________________________
flatten_55 (Flatten)         (None, 3200)              0         
_________________________________________________________________
dense_32 (Dense)             (None, 4)                 12804     
Total para

In [297]:
model2.fit(x_train, to_categorical([labels[y] for y in y_train], num_classes=4),
          validation_data=(x_test, to_categorical([labels[y] for y in y_test], num_classes=4)),
          batch_size=batch_size, epochs=num_epochs, verbose=2)

Train on 8364 samples, validate on 2789 samples
Epoch 1/10
 - 12s - loss: 0.2803 - acc: 0.9183 - val_loss: 0.1393 - val_acc: 0.9688
Epoch 2/10
 - 9s - loss: 0.1194 - acc: 0.9684 - val_loss: 0.1546 - val_acc: 0.9584
Epoch 3/10
 - 9s - loss: 0.0896 - acc: 0.9754 - val_loss: 0.1212 - val_acc: 0.9674
Epoch 4/10
 - 9s - loss: 0.0715 - acc: 0.9810 - val_loss: 0.1194 - val_acc: 0.9692
Epoch 5/10
 - 9s - loss: 0.0593 - acc: 0.9839 - val_loss: 0.1261 - val_acc: 0.9688
Epoch 6/10
 - 9s - loss: 0.0483 - acc: 0.9864 - val_loss: 0.1836 - val_acc: 0.9297
Epoch 7/10
 - 9s - loss: 0.0413 - acc: 0.9889 - val_loss: 0.1177 - val_acc: 0.9738
Epoch 8/10
 - 9s - loss: 0.0348 - acc: 0.9901 - val_loss: 0.1284 - val_acc: 0.9720
Epoch 9/10
 - 9s - loss: 0.0298 - acc: 0.9912 - val_loss: 0.1184 - val_acc: 0.9699
Epoch 10/10
 - 9s - loss: 0.0261 - acc: 0.9921 - val_loss: 0.1260 - val_acc: 0.9731


Реккурентная сеть так же стремительно уменьшала свою ошибку и растила качество
Данный алгоритм не уступает предыдущему 

## Задание 10 [8 баллов]. ULMFit

Используйте ```ULMFit``` для решения задачи классификации. Оцените результаты классификации по стандартным мерам качества и проведите анализ ошибок. Для этого рекомендуем визуализировать матрицу ошибок (confusion matrix). 

## Конец
Выполните какие-то из предыдущих заданий. Для всех заданий, кроме задания 1 требуется вычислить метрику accuracy метода.

Подведите итоги и проведите сравнение всех использованных методов. Какой из них показался вам лучше и почему?

**NB!** Задание обязательное вне зависимости от того, сколько из предыдущих пунктов вы выполнили, и дополнительных баллов не дает.


Для получения полной оценки за NLP-часть достаточно набрать **20 баллов**.

# Правила сдачи 

1. Домашняя работа должна быть выполнена в ipynb-тетрадке.
2. Сделанную тетрадку нужно отправить ассистенту (ссылка на контакты будет в вики).
3. Задание выполняется индивидуально.
4. Все вычисления должны быть снабжены пояснениями!
5. Дедлайн – 10 июня в 10.00.
